# 文字列検索

## ローリングハッシュ (Rabin-Karp)

蟻本p.332  
[Wikipedia](https://ja.wikipedia.org/wiki/%E3%83%A9%E3%83%93%E3%83%B3-%E3%82%AB%E3%83%BC%E3%83%97%E6%96%87%E5%AD%97%E5%88%97%E6%A4%9C%E7%B4%A2%E3%82%A2%E3%83%AB%E3%82%B4%E3%83%AA%E3%82%BA%E3%83%A0)

あるルールに従って生成された文字列のハッシュ値を比較することで、文字列検索を高速に行う手法。  

### アルゴリズム

文字列に対応する数列 $C=c_0, c_1, \ldots , c_{n-1}$ を考える。ただし、$n = |S|$ とする。  
この数列 $C$ を入力とするハッシュ関数を次のように定義する。

$$
H(C) = (c_0 \times b^{n-1} + c_1 \times b^{n-2} + \ldots + c_{n-1} \times b^0) \mod{m}
$$

ただし、基数 $b$ と法 $m$ は互いに素な整数 $(b<m)$ とする。  
このハッシュ関数は、文字列の $b$ 進数 $\pmod{m}$ を求める関数ともいえる。

文字列検索では文章 $T$ の各位置を起点とする長さ $n$ の部分文字列 $T_i$ に対して、ハッシュ $H(C_S) = H(C_{T_i})$ を比較する。  
愚直に計算すると計算量 $\mathcal{O}(|S||T|)$ となってしまうが、ローリングハッシュでは文字列の差分を利用して計算時間を抑える。

$S$ と $T_i = c_i, c_{i+1}, \ldots, c_{i+n}$ のハッシュ比較を考える。  
$T$ のある位置 $i=k$ で得られたハッシュ値を $T_k$ とすると、そのハッシュは

$$
H(C_k) = (c_k \times b^{n-1} + c_{k+1} \times b^{n-2} + \ldots + c_{k+(n-1)} \times b^0) \mod{m}
$$

となる。ここで、1文字隣の文字列 $i=k+1$ のハッシュは

$$
H(C_{k+1}) = (c_{k+1} \times b^n + c_{k+2} \times b^{n-1} + \ldots + c_{(k+1)+(n-1)} \times b^1) \mod{m}
$$

となる。すろと、2式からハッシュ関数の漸化式を作ることができ

$$
H(C_{k+1}) \equiv b \times H(C_k) - c_k \times b^n + c_{k+n}
$$

となる。この式の計算量は $\mathcal{O}(1)$ なので、ハッシュが衝突しなければ文字列検索全体を $\mathcal{O}(|S|+|T|)$ で行うことができる。

### 実装

$m, b$ の選び方は以下の資料を参考に、$m$ が大きな素数、$b$ が $2 \leq b < m$ となるようにした。

https://www.slideshare.net/nagisaeto/rolling-hash-149990902

In [1]:
import random


class RollingHash():

    def __init__(self, S, b=-1, m=10**9+7):
        if b < 0:
            b = random.randint(2, m-1)
        self.rh = self.make_hash(S, m, b)
        self.S = S
        self.m = m
        self.b = b
    
    def encode(self, c):
        return ord(c) - ord("a") + 1
    
    def make_hash(self, S, m, b):
        h = 0
        for s in S:
            h *= b
            h %= m
            c = self.encode(s)
            h += c
            h %= m
        return h

    def match(self, T):
        ns = len(self.S)
        nt = len(T)
        rh = self.make_hash(T[:ns], self.m, self.b)
        b_ns = pow(self.b, ns, self.m)

        i = 0
        while i <= nt - ns:
            if self.rh == rh:
                yield i
            if i < nt - ns:
                rh = rh * self.b + self.encode(T[i+ns]) - self.encode(T[i]) * b_ns
                rh %= self.m
            i += 1

In [2]:
rh = RollingHash("abc")

In [3]:
print(list(rh.match("abcaaabcbbbabc")))

[0, 5, 11]
